In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import chromedriver_autoinstaller
import time
import pandas as pd
from functools import reduce
import json
from tqdm import tqdm
import re

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("--kiosk")

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
def findkeys(node, kv):
    if isinstance(node, list):
        for i in node:
            for x in findkeys(i, kv):
                yield x
    elif isinstance(node, dict):
        if kv in node:
            yield node[kv]
        for j in node.values():
            for x in findkeys(j, kv):
                yield x

# Getting all links

In [3]:
url = 'https://www.airbnb.com.br/s/Florian%C3%B3polis-~-SC/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_lengths%5B%5D=one_week&price_filter_input_type=0&query=Florian%C3%B3polis%20-%20SC&place_id=ChIJ1zLGsk45J5URRscEagtVvIE&date_picker_type=calendar&checkin=2024-01-01&checkout=2024-01-31&source=structured_search_input_header&search_type=filter_change&price_filter_num_nights=30&room_types%5B%5D=Entire%20home%2Fapt'

In [5]:
# Initializing driver
chromedriver_autoinstaller.install()
#driver = webdriver.Chrome()

'/opt/anaconda3/lib/python3.9/site-packages/chromedriver_autoinstaller/106/chromedriver'

In [5]:
urls_list = []

# Go to first page
driver.get(url)
# Sleep to wait page render
time.sleep(30)

while(True):
    try:
        # Getting all urls from current page
        xpath = '//div[@itemprop="itemListElement"]/meta[@itemprop="url"]'
        listing = driver.find_elements(By.XPATH, xpath)
        urls_page = [i.get_attribute('content') for i in listing]
        
        current_url = driver.current_url
        
        # Saving urls into list
        urls_list.append({'page': current_url, 'list': urls_page})

        # Go to next page
        xpath_next = '//a[@aria-label="Próximo"]'
        driver.find_element(By.XPATH, xpath_next).click()
        
        # Sleep to wait page render
        time.sleep(30)
    except Exception as e:
        # When reach the last page, break
        print(e)
        break

    

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[@aria-label="Próximo"]"}
  (Session info: chrome=106.0.5249.119)
Stacktrace:
0   chromedriver                        0x00000001008dec08 chromedriver + 4402184
1   chromedriver                        0x00000001008661f3 chromedriver + 3908083
2   chromedriver                        0x0000000100513e6a chromedriver + 425578
3   chromedriver                        0x000000010054bc91 chromedriver + 654481
4   chromedriver                        0x000000010054be91 chromedriver + 654993
5   chromedriver                        0x0000000100583274 chromedriver + 881268
6   chromedriver                        0x0000000100569dcd chromedriver + 777677
7   chromedriver                        0x0000000100580f79 chromedriver + 872313
8   chromedriver                        0x0000000100569bd3 chromedriver + 777171
9   chromedriver                        0x000000010053eced chromedriver + 601325
10  chromedriver         

In [166]:
# Close driver
driver.close()

In [4]:
df = pd.DataFrame(urls_list)
df

NameError: name 'urls_list' is not defined

In [6]:
#df.to_pickle('listing.pck')
df = pd.read_pickle('listing.pck')
df

,page,list
0,https://www.airbnb.com.br/s/Florian%C3%B3polis...,[www.airbnb.com.br/rooms/28402863?check_in=202...
1,https://www.airbnb.com.br/s/Florian%C3%B3polis...,[www.airbnb.com.br/rooms/21833478?check_in=202...
2,https://www.airbnb.com.br/s/Florian%C3%B3polis...,[www.airbnb.com.br/rooms/30744359?check_in=202...
3,https://www.airbnb.com.br/s/Florian%C3%B3polis...,[www.airbnb.com.br/rooms/49093689?check_in=202...
4,https://www.airbnb.com.br/s/Florian%C3%B3polis...,[www.airbnb.com.br/rooms/48266519?check_in=202...
5,https://www.airbnb.com.br/s/Florian%C3%B3polis...,[www.airbnb.com.br/rooms/15801255?check_in=202...
6,https://www.airbnb.com.br/s/Florian%C3%B3polis...,[www.airbnb.com.br/rooms/10242695?check_in=202...
7,https://www.airbnb.com.br/s/Florian%C3%B3polis...,[www.airbnb.com.br/rooms/48498927?check_in=202...
8,https://www.airbnb.com.br/s/Florian%C3%B3polis...,[www.airbnb.com.br/rooms/49977748?check_in=202...
9,https://www.airbnb.com.br/s/Florian%C3%B3polis...,[www.airbnb.com.br/rooms/47418392?check_in=202...


In [7]:
links = reduce(lambda x,y: x+y, df['list'].values)
links = list(map(lambda x: 'https://' + x, links))
len(links)

300

In [8]:
months = ['0' + str(i) if len(str(i)) < 2 else str(i) for i in range (1, 13)]
day = '01'
years = ['2023', '2024', '2026']

In [9]:
dates_ranges = []

for y in years:
    for m in months:
        date = y + '-' + m + '-' + day
        dates_ranges.append(date)
        
dates_ranges    

['2023-01-01',
 '2023-02-01',
 '2023-03-01',
 '2023-04-01',
 '2023-05-01',
 '2023-06-01',
 '2023-07-01',
 '2023-08-01',
 '2023-09-01',
 '2023-10-01',
 '2023-11-01',
 '2023-12-01',
 '2024-01-01',
 '2024-02-01',
 '2024-03-01',
 '2024-04-01',
 '2024-05-01',
 '2024-06-01',
 '2024-07-01',
 '2024-08-01',
 '2024-09-01',
 '2024-10-01',
 '2024-11-01',
 '2024-12-01',
 '2026-01-01',
 '2026-02-01',
 '2026-03-01',
 '2026-04-01',
 '2026-05-01',
 '2026-06-01',
 '2026-07-01',
 '2026-08-01',
 '2026-09-01',
 '2026-10-01',
 '2026-11-01',
 '2026-12-01']

In [10]:
str_ranges = ['check_in=' + dates_ranges[i - 1] +  '&check_out=' + dates_ranges[i] for i in range(1, len(dates_ranges))]
str_ranges

['check_in=2023-01-01&check_out=2023-02-01',
 'check_in=2023-02-01&check_out=2023-03-01',
 'check_in=2023-03-01&check_out=2023-04-01',
 'check_in=2023-04-01&check_out=2023-05-01',
 'check_in=2023-05-01&check_out=2023-06-01',
 'check_in=2023-06-01&check_out=2023-07-01',
 'check_in=2023-07-01&check_out=2023-08-01',
 'check_in=2023-08-01&check_out=2023-09-01',
 'check_in=2023-09-01&check_out=2023-10-01',
 'check_in=2023-10-01&check_out=2023-11-01',
 'check_in=2023-11-01&check_out=2023-12-01',
 'check_in=2023-12-01&check_out=2024-01-01',
 'check_in=2024-01-01&check_out=2024-02-01',
 'check_in=2024-02-01&check_out=2024-03-01',
 'check_in=2024-03-01&check_out=2024-04-01',
 'check_in=2024-04-01&check_out=2024-05-01',
 'check_in=2024-05-01&check_out=2024-06-01',
 'check_in=2024-06-01&check_out=2024-07-01',
 'check_in=2024-07-01&check_out=2024-08-01',
 'check_in=2024-08-01&check_out=2024-09-01',
 'check_in=2024-09-01&check_out=2024-10-01',
 'check_in=2024-10-01&check_out=2024-11-01',
 'check_in

In [11]:
links[0]

'https://www.airbnb.com.br/rooms/28402863?check_in=2024-01-01&check_out=2024-01-31&previous_page_section_name=1000&display_extensions%5B0%5D=MONTHLY_STAYS'

# Crawling informations

In [12]:
complete_links = [[l.replace(r'check_in=2024-01-01&check_out=2024-01-31', range_) for l in links] for range_ in str_ranges]
complete_links = reduce(lambda x,y: x+y, complete_links)
complete_links[0:10]

['https://www.airbnb.com.br/rooms/28402863?check_in=2023-01-01&check_out=2023-02-01&previous_page_section_name=1000&display_extensions%5B0%5D=MONTHLY_STAYS',
 'https://www.airbnb.com.br/rooms/47933102?check_in=2023-01-01&check_out=2023-02-01&previous_page_section_name=1000&display_extensions%5B0%5D=MONTHLY_STAYS',
 'https://www.airbnb.com.br/rooms/37463536?check_in=2023-01-01&check_out=2023-02-01&previous_page_section_name=1000&display_extensions%5B0%5D=MONTHLY_STAYS',
 'https://www.airbnb.com.br/rooms/31143465?check_in=2023-01-01&check_out=2023-02-01&previous_page_section_name=1000&display_extensions%5B0%5D=MONTHLY_STAYS',
 'https://www.airbnb.com.br/rooms/27789679?check_in=2023-01-01&check_out=2023-02-01&previous_page_section_name=1000&display_extensions%5B0%5D=MONTHLY_STAYS',
 'https://www.airbnb.com.br/rooms/52364562?check_in=2023-01-01&check_out=2023-02-01&previous_page_section_name=1000&display_extensions%5B0%5D=MONTHLY_STAYS',
 'https://www.airbnb.com.br/rooms/17963094?check_in=

In [13]:
links_2023 = [cp for cp in complete_links if '2023-' in cp]
links_2023[len(links_2023) - 6:len(links_2023) -1]

['https://www.airbnb.com.br/rooms/31053056?check_in=2023-12-01&check_out=2024-01-01&previous_page_section_name=1000&display_extensions%5B0%5D=MONTHLY_STAYS',
 'https://www.airbnb.com.br/rooms/27467426?check_in=2023-12-01&check_out=2024-01-01&previous_page_section_name=1000&display_extensions%5B0%5D=MONTHLY_STAYS',
 'https://www.airbnb.com.br/rooms/23606721?check_in=2023-12-01&check_out=2024-01-01&previous_page_section_name=1000&display_extensions%5B0%5D=MONTHLY_STAYS',
 'https://www.airbnb.com.br/rooms/52896730?check_in=2023-12-01&check_out=2024-01-01&previous_page_section_name=1000&display_extensions%5B0%5D=MONTHLY_STAYS',
 'https://www.airbnb.com.br/rooms/46081728?check_in=2023-12-01&check_out=2024-01-01&previous_page_section_name=1000&display_extensions%5B0%5D=MONTHLY_STAYS']

In [14]:
results = []
errors = []

k = -1

for url in tqdm(links_2023):
    # Tries counter
    j = 0
    
    k = k + 1
    
    if (k % 10) == 0:
        print('Errors total: ', len(errors))
    
    while(True):
        try:
            #chromedriver_autoinstaller.install()
            j = j + 1
            
            time.sleep(2)
            
            driver = webdriver.Chrome(options = options)

            time.sleep(2)

            driver.get(url)

            time.sleep(20)

            title = driver.find_element(By.XPATH, '//div[@data-section-id="TITLE_DEFAULT"]/section/div/span/h1').text
            rating = driver.find_element(By.XPATH, '//span[@class="_17p6nbba"]').text
            location = driver.find_element(By.XPATH, '//span[@class="_9xiloll"]').text
            
            try:
                price = ' '.join([i.text for i in driver.find_elements(By.XPATH, '//div//span[@class="_tyxjp1"]')])
            except:
                price = ' '.join(re.findall('R\$(\d+\.\d+)', driver.execute_script("return document.documentElement.innerHTML;")))
                
            description = driver.find_element(By.XPATH, '//span[@class="ll4r2nl dir dir-ltr"]').text
            guests = driver.find_element(By.XPATH, '//li[@class="l7n4lsf dir dir-ltr"]').text

            atributes = driver.find_elements(By.XPATH, '//li[@class="l7n4lsf dir dir-ltr"]')
            atributes = [i.text for i in atributes]

            am = json.loads(driver.find_element(By.XPATH, '//script[@id="data-deferred-state"]').get_attribute('innerHTML'))


            amenities_list = [i for i in am['niobeMinimalClientData'][0][1]['data']['presentation']['stayProductDetailPage']['sections']['sections'] if 'previewAmenitiesGroups' in i['section']][0]['section']['seeAllAmenitiesGroups']
            amenities = [[a['title'] for a in al['amenities']] for al in amenities_list]
            amenities_list = reduce(lambda x,y: x+y, amenities)

            ratings = driver.find_elements(By.XPATH, '//div[@class="_1s11ltsf"]//div//span[@class="_4oybiu"]')
            ratings = [i.text for i in ratings]

            lat = str(list(findkeys(am, 'lat'))[0]) if len(list(findkeys(am, 'lat'))) > 0 else ''
            lng = str(list(findkeys(am, 'lng'))[0]) if len(list(findkeys(am, 'lng'))) > 0 else ''

            data = {
                    'url': url,
                    'title': title,
                   'rating': rating,
                   'location': location,
                   'price': price,
                   'description': description,
                   'guests': guests,
                   'atributes': atributes,
                   'amenities_list': amenities_list,
                   'ratings': ratings,
                   'lat': lat,
                   'lng': lng}

            results.append(data)

            driver.close()
            break

        except Exception as e:
            print(e)
            # Exceeded 5 tries, save link and go next
            if j >= 5:
                errors.append(url)
                break

            continue

  0%|          | 0/3600 [00:00<?, ?it/s]

Errors total:  0


  0%|          | 10/3600 [05:20<31:42:49, 31.80s/it]

Errors total:  0


  1%|          | 20/3600 [10:42<31:57:46, 32.14s/it]

...

 49%|████▉     | 1770/3600 [20:41:13<253:38:52, 498.98s/it]

Errors total:  48


 49%|████▉     | 1779/3600 [21:29:17<21:59:44, 43.48s/it]  
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/dl/144dv_xn32j603jj7l33r0h40000gn/T/ipykernel_20095/758320156.py", line 28, in <module>
    time.sleep(20)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 2064, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/opt/anaconda3/lib/python3

TypeError: object of type 'NoneType' has no len()

In [41]:
driver.close()

In [18]:
df = pd.DataFrame(results)
df.to_csv('2023_1_data.csv', index = False)

In [19]:
df_errors = pd.DataFrame(errors)
df_errors.to_csv('2023_1_errors.csv', index = False)

In [20]:
pd.read_csv('2023_1_data.csv')

,url,title,rating,location,price,description,guests,atributes,amenities_list,ratings,lat,lng
0,https://www.airbnb.com.br/rooms/28402863?check...,Studio acolhedor na Lagoa da Conceição,"4,95 ·","Lagoa da Conceição, Santa Catarina, Brasil",R$5.480,Nosso studio com design rústico é perfeito par...,2 hóspedes,"['2 hóspedes', ' · 1 quarto', ' · 1 cama', ' ·...","['Vista para o jardim', 'Secador de cabelo', '...","['4,9', '5,0', '5,0', '4,9', '5,0', '4,9']",-27.60651,-48.47320
1,https://www.airbnb.com.br/rooms/47933102?check...,Apt completo com ótima localização em Floripa!,"5,0 ·","Trindade, Santa Catarina, Brasil",R$5.201,Hospede-se em Floripa em uma acomodação comple...,4 hóspedes,"['4 hóspedes', ' · 1 quarto', ' · 1 cama', ' ·...","['Secador de cabelo', 'Máquina de Lavar', 'Bás...","['5,0', '5,0', '5,0', '5,0', '4,9', '4,9']",-27.58670,-48.52479
2,https://www.airbnb.com.br/rooms/37463536?check...,"Linda Kit em frente UFSC, entre praias norte e...","5,0 ·","Trindade, Santa Catarina, Brasil",R$5.200,"Apartamento em bairro super bem localizado, co...",4 hóspedes,"['4 hóspedes', ' · 1 quarto', ' · 2 camas', ' ...","['Vista para o parque', 'Produtos de limpeza',...","['5,0', '4,9', '5,0', '5,0', '5,0', '5,0']",-27.59710,-48.52400
3,https://www.airbnb.com.br/rooms/31143465?check...,Linda casa confortável em Florianópolis,"4,83 ·","Ponta das Canas, Santa Catarina, Brasil",R$10.430,"Casa com 3 dormitórios, sala de estar,sala de ...",8 hóspedes,"['8 hóspedes', ' · 3 quartos', ' · 3 camas', '...","['Água quente', 'Máquina de Lavar', 'Básico', ...","['4,5', '4,8', '5,0', '4,8', '5,0', '4,9']",-27.39643,-48.42749
4,https://www.airbnb.com.br/rooms/27789679?check...,Apto no Centro ideal para lazer ou trabalho,"4,88 ·","Centro, Santa Catarina, Brasil",R$1.980,Apartamento de um quarto com sala de estar/jan...,2 hóspedes,"['2 hóspedes', ' · 1 quarto', ' · 1 cama', ' ·...","['Secador de cabelo', 'Produtos de limpeza', '...","['4,8', '4,9', '5,0', '4,9', '5,0', '4,8']",-27.59888,-48.54935
...,...,...,...,...,...,...,...,...,...,...,...,...
1726,https://www.airbnb.com.br/rooms/29564208?check...,Lindo Apartamento Pé na Areia⭐⭐⭐⭐⭐ Praia Campeche,"4,95 ·","Campeche Leste, Santa Catarina, Brasil",R$9.180,- Apartamento 2 dormitórios com ar condicionad...,6 hóspedes,"['6 hóspedes', ' · 2 quartos', ' · 5 camas', '...","['Vista para o jardim', 'Vista para o mar', 'V...","['4,9', '5,0', '5,0', '5,0', '4,9', '4,8']",-27.68995,-48.48464
1727,https://www.airbnb.com.br/rooms/47098945?check...,Studio S: lazer/trabalho/moradia. Próx. UFSC.,"5,0 ·","Saco dos Limões, Santa Catarina, Brasil",R$1.980,"Loft novo, mobiliado e equipado, com estação d...",2 hóspedes,"['2 hóspedes', ' · Estúdio', ' · 1 cama', ' · ...","['Secador de cabelo', 'Máquina de lavar no pré...","['5,0', '5,0', '3,3', '5,0', '4,0', '4,7']",-27.61750,-48.52640
1728,https://www.airbnb.com.br/rooms/38484414?check...,Campeche Apartment 4,"4,84 ·","Florianópolis , Santa Catarina, Brasil",R$2.370,NaN,2 hóspedes,"['2 hóspedes', ' · Estúdio', ' · 1 cama', ' · ...","['Vista para o pátio', 'Água quente', 'Básico'...","['', '', '', '', '', '']",-27.69095,-48.49015
1729,https://www.airbnb.com.br/rooms/38457832?check...,Confortável apto para férias na praia!,"4,50 ·","Ingleses Norte, Santa Catarina, Brasil",R$6.723,"Aproveite toda a comodidade e conforto, perto...",6 hóspedes,"['6 hóspedes', ' · 2 quartos', ' · 5 camas', '...","['Secador de cabelo', 'Água quente', 'Gel de b...","['4,9', '4,5', '4,9', '4,9', '4,9', '4,8']",-27.42798,-48.40519
